In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer, MQRNNEstimator, TemporalFusionTransformerEstimator
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory

from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [2]:
mx.context.num_gpus()

0

In [3]:
# df = pd.read_csv('Merged-update_hourly.csv')
df = pd.read_csv('../../../data/TFT_energy.csv')

In [4]:
df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,wind_deg_Bilbao,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump
0,2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,...,229.0,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0
1,2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,...,224.0,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0
2,2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,...,225.0,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0
3,2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,...,221.0,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0
4,2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,...,224.0,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,57.0,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,140.0,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,120.0,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0


In [5]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'time'], infer_datetime_format=True)

In [6]:
df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump,timestamp
0,2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,...,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0,2015-01-01 00:00:00+00:00
1,2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,...,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0,2015-01-01 01:00:00+00:00
2,2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,...,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0,2015-01-01 02:00:00+00:00
3,2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,...,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0,2015-01-01 03:00:00+00:00
4,2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,...,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0,2015-01-01 04:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0,2018-12-31 18:00:00+00:00
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0,2018-12-31 19:00:00+00:00
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0,2018-12-31 20:00:00+00:00
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0,2018-12-31 21:00:00+00:00


In [7]:
df.columns

Index(['time', 'price', 'price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump',
       'timestamp'],
      dtype='object')

In [8]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [9]:
train_df.shape

(28050, 28)

In [10]:
test_df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump,timestamp
28050,2018-03-14 18:00:00+00:00,50.95,37.61,1068.0,5319.0,35044.0,0.0,345.0,102.0,1006.0,...,40.0,1849.0,220.0,1.0,1.0,8.0,14265.0,6.0,761.0,2018-03-14 18:00:00+00:00
28051,2018-03-14 19:00:00+00:00,59.73,48.21,1126.0,5292.0,36230.0,0.0,351.0,101.0,1005.0,...,40.0,1821.0,220.0,4.0,2.0,8.0,13306.0,5.0,149.0,2018-03-14 19:00:00+00:00
28052,2018-03-14 20:00:00+00:00,50.89,41.11,1054.0,4318.0,35065.0,0.0,351.0,103.0,1005.0,...,40.0,1853.0,220.0,1.0,2.0,5.0,12970.0,7.0,168.0,2018-03-14 20:00:00+00:00
28053,2018-03-14 21:00:00+00:00,45.94,36.72,963.0,4047.0,32328.0,0.0,350.0,104.0,1004.0,...,20.0,1853.0,210.0,1.0,1.0,1.0,13090.0,3.0,811.0,2018-03-14 21:00:00+00:00
28054,2018-03-14 22:00:00+00:00,30.59,21.20,853.0,3726.0,29528.0,0.0,329.0,95.0,1004.0,...,0.0,1826.0,220.0,0.0,8.0,5.0,13408.0,6.0,2007.0,2018-03-14 22:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0,2018-12-31 18:00:00+00:00
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0,2018-12-31 19:00:00+00:00
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0,2018-12-31 20:00:00+00:00
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0,2018-12-31 21:00:00+00:00


In [11]:
train_ds = PandasDataset(
    train_df,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
    freq = 'H'
)

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [12]:
test_ds = PandasDataset(
    test_df,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
    freq = 'H'
)

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


#### training

In [13]:
K = 24
n_hours = 24*3

In [14]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=150,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-06)

estimator = TemporalFusionTransformerEstimator(
    prediction_length=K,
    freq="H",
#     use_feat_dynamic_real=True,
    batch_size=32,
    trainer=trainer,
    dropout_rate=0.3,
    hidden_dim=64,
    num_heads=2,
)

model = estimator.train(train_ds)

100%|████████████████████████████████████████████████| 100/100 [02:11<00:00,  1.32s/it, epoch=130/150, avg_epoch_loss=0.586]


100%|█████████████████████████████████████████████████| 100/100 [02:14<00:00,  1.35s/it, epoch=150/150, avg_epoch_loss=0.55]


#### predicted y

In [15]:
quantiles = ['0.5', '0.1', '0.9']
q50 = 0

In [16]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].forecast_array
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


0
[58.42610168 61.05018234 59.3572731  55.68545914 53.04849243 51.79339218
 47.31964111 44.29513931 42.47773361 42.14053345 42.79728317 44.08453369
 45.74670029 48.347332   50.19827271 50.54433823 50.65386581 50.97834396
 51.45471191 51.05072021 49.46831894 47.96995926 49.11548233 56.09317017]
(24,)
------
1
[58.42610168 61.05018234 59.3572731  55.68545914 53.04849243 51.79339218
 47.31964111 44.29513931 42.47773361 42.14053345 42.79728317 44.08453369
 45.74670029 48.347332   50.19827271 50.54433823 50.65386581 50.97834396
 51.45471191 51.05072021 49.46831894 47.96995926 49.11548233 56.09317017
 62.89131546 60.84146118 56.88270187 53.99438477 52.71469116 48.10300827
 44.918396   43.06099319 42.67730713 43.32185364 44.63154221 46.28011703
 48.85124207 50.65396118 50.98904419 51.08355713 51.45494461 51.84636307
 51.38153076 49.76104736 48.21905899 49.38063431 56.60797119 65.34584808]
(48,)
------
2
[58.42610168 61.05018234 59.3572731  55.68545914 53.04849243 51.79339218
 47.31964111 44.2

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index repr

[58.42610168 61.05018234 59.3572731  55.68545914 53.04849243 51.79339218
 47.31964111 44.29513931 42.47773361 42.14053345 42.79728317 44.08453369
 45.74670029 48.347332   50.19827271 50.54433823 50.65386581 50.97834396
 51.45471191 51.05072021 49.46831894 47.96995926 49.11548233 56.09317017
 62.89131546 60.84146118 56.88270187 53.99438477 52.71469116 48.10300827
 44.918396   43.06099319 42.67730713 43.32185364 44.63154221 46.28011703
 48.85124207 50.65396118 50.98904419 51.08355713 51.45494461 51.84636307
 51.38153076 49.76104736 48.21905899 49.38063431 56.60797119 65.34584808
 63.05842209 58.57800293 54.63764954 52.59281921 47.21111679 43.47208786
 41.43250656 41.07848358 41.76534653 43.28700638 45.24622726 48.17398071
 50.64644241 51.26031113 51.57492065 52.31861877 52.546772   51.63942337
 49.51361084 47.50815964 48.95997238 57.63037109 67.87573242 71.45159912
 60.68493652 55.80758667 52.57512283 47.00341415 43.63616943 41.73158646
 41.18156433 41.55330276 42.82719421 44.69783401 47

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index repr

...

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index repr

[58.42610168 61.05018234 59.3572731  ... 68.4776535  67.13859558
 65.63357544]
(165768,)
------
6907
[58.42610168 61.05018234 59.3572731  ... 67.26873016 66.00051117
 64.74030304]
(165792,)
------
6908
[58.42610168 61.05018234 59.3572731  ... 66.56365967 65.179039
 63.91592026]
(165816,)
------
6909
[58.42610168 61.05018234 59.3572731  ... 63.05657959 61.79602051
 60.87921524]
(165840,)
------
6910
[58.42610168 61.05018234 59.3572731  ... 60.66914749 59.9011116
 59.77549362]
(165864,)
------
6911
[58.42610168 61.05018234 59.3572731  ... 59.34597778 59.25891495
 60.51258469]
(165888,)
------
6912


/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index repr

[58.42610168 61.05018234 59.3572731  ... 59.66936111 61.20672989
 64.85765076]
(165912,)
------
6913
[58.42610168 61.05018234 59.3572731  ... 60.56472778 64.49745941
 65.04147339]
(165936,)
------
6914
[58.42610168 61.05018234 59.3572731  ... 62.59736252 63.23681259
 62.24716187]
(165960,)
------
6915
[58.42610168 61.05018234 59.3572731  ... 62.2665863  61.45143509
 60.23359299]
(165984,)
------
6916
[58.42610168 61.05018234 59.3572731  ... 60.72395325 59.39021301
 59.30892944]
(166008,)
------
6917
[58.42610168 61.05018234 59.3572731  ... 58.77492142 57.70582581
 57.56372452]
(166032,)
------


/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index repr

In [17]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(6918, 24)

In [18]:
test_ds_predict_q50_supervised[0:3]

array([[58.42610168, 61.05018234, 59.3572731 , 55.68545914, 53.04849243,
        51.79339218, 47.31964111, 44.29513931, 42.47773361, 42.14053345,
        42.79728317, 44.08453369, 45.74670029, 48.347332  , 50.19827271,
        50.54433823, 50.65386581, 50.97834396, 51.45471191, 51.05072021,
        49.46831894, 47.96995926, 49.11548233, 56.09317017],
       [62.89131546, 60.84146118, 56.88270187, 53.99438477, 52.71469116,
        48.10300827, 44.918396  , 43.06099319, 42.67730713, 43.32185364,
        44.63154221, 46.28011703, 48.85124207, 50.65396118, 50.98904419,
        51.08355713, 51.45494461, 51.84636307, 51.38153076, 49.76104736,
        48.21905899, 49.38063431, 56.60797119, 65.34584808],
       [63.05842209, 58.57800293, 54.63764954, 52.59281921, 47.21111679,
        43.47208786, 41.43250656, 41.07848358, 41.76534653, 43.28700638,
        45.24622726, 48.17398071, 50.64644241, 51.26031113, 51.57492065,
        52.31861877, 52.546772  , 51.63942337, 49.51361084, 47.50815964,
  

#### actual y

In [19]:
from helper import series_to_supervised, stage_series_to_supervised

In [21]:
test_df_supervised = series_to_supervised(test_df[['price']], n_hours, K)

In [22]:
test_df_supervised.iloc[:, -K:].shape

(6918, 24)

In [23]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [24]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),...,var1(t+14),var1(t+15),var1(t+16),var1(t+17),var1(t+18),var1(t+19),var1(t+20),var1(t+21),var1(t+22),var1(t+23)
28122,63.57,73.69,67.93,64.33,59.28,55.41,53.16,49.20,48.17,46.76,...,46.77,46.58,46.62,46.13,45.39,45.04,41.33,29.18,36.40,47.22
28123,73.69,67.93,64.33,59.28,55.41,53.16,49.20,48.17,46.76,47.25,...,46.58,46.62,46.13,45.39,45.04,41.33,29.18,36.40,47.22,53.73
28124,67.93,64.33,59.28,55.41,53.16,49.20,48.17,46.76,47.25,46.73,...,46.62,46.13,45.39,45.04,41.33,29.18,36.40,47.22,53.73,58.17
28125,64.33,59.28,55.41,53.16,49.20,48.17,46.76,47.25,46.73,45.66,...,46.13,45.39,45.04,41.33,29.18,36.40,47.22,53.73,58.17,58.79
28126,59.28,55.41,53.16,49.20,48.17,46.76,47.25,46.73,45.66,48.20,...,45.39,45.04,41.33,29.18,36.40,47.22,53.73,58.17,58.79,55.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,73.35,73.64,71.92,69.49,68.40,60.32,57.73,57.66,57.04,57.48,...,72.12,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02
35036,73.64,71.92,69.49,68.40,60.32,57.73,57.66,57.04,57.48,61.13,...,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16
35037,71.92,69.49,68.40,60.32,57.73,57.66,57.04,57.48,61.13,60.70,...,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16,74.30
35038,69.49,68.40,60.32,57.73,57.66,57.04,57.48,61.13,60.70,66.64,...,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16,74.30,69.89


#### performance on test set

In [25]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 3.79
RMSE = 5.615
